In [ ]:
import os 
import cv2
import numpy as np
from skimage import transform

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
rad2deg = 180 / np.pi
figsize = (12, 8)

# indices, `DP' = data point
DP1 = 0
DP2 = 1
X = 0
Y = 1

save_figures = False

In [ ]:
path = '/scratch/ssd/cciw/sample_data/'
images = os.listdir(path)

#file = '1340_2016-07-11_1_GLN_3447.JPG' # 5/5
#file = '1340_2016-07-11_2_GLN_3450.JPG' # 4/5
file = '1340_2016-07-11_3_GLN_3452.JPG' # ?/5
#file = '1349_2016-07-06_2_GLN_3061.JPG'  # 3/5, step 3 lines are offset, probably because portrait vs. landscape mode
#img = '1342_2016-07-07_2_GLN_3182.JPG'
#img = 'sudoku.png'
outpath = file.split('.')[0]

In [ ]:
f = os.path.join(path, file)
im = cv2.imread(f)
im.shape  # (height, width) # (3776, 6720, 3)

In [ ]:
ds = 8  # naive downsampling by factor of 8
plt.figure(figsize=figsize)
plt.imshow(im[::ds, ::ds])

In [ ]:
'''
@param threshold Accumulator threshold parameter. Only those lines are returned that get enough
       votes ( \f$>\texttt{threshold}\f$ ).
@param minLineLength Minimum line length. Line segments shorter than that are rejected.
@param maxLineGap Maximum allowed gap between points on the same line to link them.
'''
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

edges = cv2.Canny(gray, 20, 400, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator in pixels.
rho = 1  
# @param theta Angle resolution of the accumulator in radians.
theta = np.pi / 90  # note, with 180 getting too many overlapping lines.

lines  =  cv2.HoughLines(edges, rho, theta, threshold=100)
N = 1
coords = np.zeros((N, 2, 2)).astype('int') # points, start/end, x/y
angles = np.zeros(N)
for i in range(len(lines[:N])):
    for r, theta in lines[i]:
        # theta is normal to the line wrt x-axis, so we subtract 90
        if r > 0:
            angles[i] = 90 - theta * 180 / np.pi
        else: 
            angles[i] = 270 - theta * 180 / np.pi
        print(i, '%.2f' % angles[i])
        a = np.cos(theta)
        b = np.sin(theta)
        x0, y0 = a * r, b * r
        x1 = int(x0 + img_w * -b)
        y1 = int(y0 + img_w *  a)
        x2 = int(x0 - img_w * -b)
        y2 = int(y0 - img_w *  a)
        #coords.append([[x1, y1], [x2, y2]])
        coords[i, DP1, X] = x1
        coords[i, DP1, Y] = y1
        coords[i, DP2, X] = x2
        coords[i, DP2, Y] = y2
        print(i, (x1, y1), (x2, y2))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
plt.figure(figsize=figsize)
plt.imshow(img)
#plt.xlabel('x')
#plt.ylabel('y')
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-1.png')
plt.show()

## Probabilistic HoughLines

In [ ]:
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# @param canny_threshold1 Histeresis threshold 1
# @param canny_threshold2
canny_thresh1 = 20
canny_thresh2 = 400

# run the Canny edge detector on the rotated gray scale image
edges = cv2.Canny(gray, threshold1=canny_thresh1, threshold2=canny_thresh2, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator (pixels).
rho = 1  

# @param theta Angle resolution of the accumulator (radians).
theta = np.pi / 90

# @param threshold Accumulator threshold, return lines with more than threshold of votes. (intersection points)
threshold = 100

# @param minLineLength Minimum line length. Line segments shorter than that are rejected. (pixels)
mLL = 300

# @param maxLineGap Maximum allowed gap between points on the same line to link them. (pixels)
mLG = 100

# run the probabilistic hough lines transform
linesP = cv2.HoughLinesP(edges, rho, theta, threshold=threshold, minLineLength=mLL, maxLineGap=mLG)

# @param lines The extremes of the detected lines if any (<N_LINES_FOUND>, 1, x_0, y_0, x_1, y_1). (pixels)
print('Found %d lines' % len(linesP))

N = 8  # top N results to draw
if linesP is not None:
    for i in range(len(linesP[:N])):
        l = linesP[i][0]
        cv2.line(img, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3, cv2.LINE_AA)
plt.figure(figsize=figsize)
plt.imshow(img)
#plt.xlabel('x')
#plt.ylabel('y')
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()

In [ ]:
# line below origin, rho positive, angle less than 180
# line above origin, rho negative, angle less than 180
lines = cv2.HoughLines(edges, rho, theta, threshold=100)

if lines[0][0][1] > 0:
    rot_deg = 90 - lines[0][0][1] * rad2deg
else:
    rot_deg = 0
t = rot_deg / rad2deg

In [ ]:
coordsP = np.zeros((np.minimum(N, len(linesP)), 2, 2)).astype('int') # points, start/end, x/y

for i in range(len(linesP[:N])):
    l = linesP[i][0]
    coordsP[i, DP1, X] = l[0] # x1
    coordsP[i, DP1, Y] = l[1] # y1
    coordsP[i, DP2, X] = l[2] # x2
    coordsP[i, DP2, Y] = l[3] # y2
    
R = np.array([[np.cos(-t), -np.sin(-t)],
              [np.sin(-t),  np.cos(-t)]])
coords = np.dot(coordsP, R)

correction_factor = coordsP[0, DP1, Y] - coords[0, DP1, Y]

for i in range(len(linesP[:N])):
    coords[i, DP1, Y] += correction_factor
    coords[i, DP2, Y] += correction_factor

In [ ]:
#plt.figure(figsize=(12, 10))
center = (coordsP[0, DP1, X], coordsP[0, DP1, Y])
imgr = transform.rotate(np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8), -rot_deg, center=center, resize=False)
imgr = (255 * imgr).astype(np.uint8)
clean_img = imgr.copy()
plt.imshow(imgr[:, :])
plt.show()

In [ ]:
for i in range(len(linesP[:N])):
    cv2.line(imgr, (int(coords[i, DP1, X]),
                    int(coords[i, DP1, Y])),
                   (int(coords[i, DP2, X]),
                    int(coords[i, DP2, Y])),
             (255, 0, 0), 3, cv2.LINE_AA)
plt.figure(figsize=figsize)
plt.imshow(imgr)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-3.png')

# Find Intersection Points

In [ ]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

'''print(line_intersection((coords[0, DP1], coords[0, DP2]), 
                           (coords[1, DP1], coords[0, DP2])))'''

In [ ]:
corners = []
for i in range(len(coords)):
    for j in range(i + 1, len(coords)):
        intersection = line_intersection((coords[i, DP1], coords[i, DP2]), 
                                         (coords[j, DP1], coords[j, DP2]))
        # check if intersection is a valid corner
        if intersection is not None:
            cond1 = 0 < intersection[X] and intersection[X] < img_w
            cond2 = 0 < intersection[Y] and intersection[Y] < img_h
            if cond1 and cond2:
                corners.append(intersection)
corners = np.asarray(corners)
centroid = corners.mean(axis=0, keepdims=True)[0]
corner_dist = np.linalg.norm(corners - centroid, axis=1)
indices = np.argsort(corner_dist)
crop = corners[indices][:4].astype('int')

In [ ]:
#fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
#fig, ax2 = plt.subplots(1, 1, figsize, sharex=True)
fig = plt.figure(figsize=figsize)
ax2 = fig.gca()
#ax1.scatter(corners[:, 0], corners[:, 1], c='r')
ax2.scatter(corners[:, 0], corners[:, 1], c='b')
ax2.scatter(centroid[0], centroid[1], c='k')
ax2.scatter(crop[:, 0], crop[:, 1], c='pink')
#ax1.set_xlim(0, img_w)
#ax1.set_ylim(img_h, 0)
ax2.imshow(imgr)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-4.png')
#ax1.set_aspect('equal')

In [ ]:
buffer = 10
plt.figure(figsize=(6, 6))
cropped_img = clean_img[crop[:, 1].min() + buffer:crop[:, 1].max() - buffer, 
                        crop[:, 0].min() + buffer:crop[:, 0].max() - buffer, :]
plt.imshow(cropped_img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-5.png')